In [19]:
import ast
import pandas as pd
import os
from itertools import product

performance_matrix_path = "./prediction_res/performance_matrix.csv"
power_cap_res_path = "./ecp_power_cap_res/2_dual_cap/"

# Applications of interest
apps = ['miniGAN', 'sw4lite', 'bert_large', 'UNet', 'Resnet50', 'lammps', 'gromacs', 'NAMD']
apps = ['bert_large']

# Load performance matrix
performance_matrix = pd.read_csv(performance_matrix_path, index_col=0)

# Dictionary to store results
edp_results = {}

cpu_caps = [200, 190, 180, 170, 160, 150, 140, 130, 120]
gpu_caps = [250, 240, 230, 220, 210, 200, 190, 180, 170, 160, 150]
power_cap_pairs = list(product(cpu_caps, gpu_caps))

for app in apps:
    app_path = os.path.join(power_cap_res_path, app)
    

    min_edp = float('inf')
    best_power_pair = None

    for power_pair in power_cap_pairs:
        cpu_power, gpu_power = power_pair  # Unpack tuple

        # # Ensure the power pair exists in the performance matrix
        # if power_pair not in performance_matrix.index: 
        #     print(power_pair)
            
        # if app not in performance_matrix.columns:
        #     print(app)
            

        # Get normalized performance
        power_pair = f"({cpu_power}, {gpu_power})"

        norm_performance = performance_matrix.loc[power_pair, app]
        # print(norm_performance)

        # Compute normalized execution time
        norm_exec_time = 1 / norm_performance

        # Load CPU and GPU power files
        cpu_power_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_cpu_power.csv")
        gpu_power_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_gpu_metrics.csv")

        if not os.path.exists(cpu_power_file) or not os.path.exists(gpu_power_file):
            print("hello")

        cpu_data = pd.read_csv(cpu_power_file)
        gpu_data = pd.read_csv(gpu_power_file)

        # Compute CPU energy consumption
        cpu_energy = (cpu_data["Package Power (W)"].mean() + cpu_data["DRAM Power (W)"].mean()) * norm_exec_time

        # Compute GPU energy consumption
        gpu_energy = gpu_data["Power (W)"].mean() * norm_exec_time

        # Total energy consumption
        total_energy = cpu_energy + gpu_energy

        # Compute EDP
        edp = total_energy * norm_exec_time

        # Update best power pair
        if edp < min_edp:
            min_edp = edp
            best_power_pair = (cpu_power, gpu_power)

    # Step 3: Compute EDP for actual performance using the selected power cap pair and baseline (200_250)
    if best_power_pair:
        cpu_power, gpu_power = best_power_pair

        # Baseline power cap (200_250)
        baseline_cpu_file = os.path.join(app_path, "200_250_cpu_power.csv")
        baseline_gpu_file = os.path.join(app_path, "200_250_gpu_metric.csv")

        if os.path.exists(baseline_cpu_file) and os.path.exists(baseline_gpu_file):
            baseline_cpu_data = pd.read_csv(baseline_cpu_file)
            baseline_gpu_data = pd.read_csv(baseline_gpu_file)

            # Execution time from the last row of Time (s)
            baseline_exec_time = baseline_cpu_data["Time (s)"].iloc[-1]

            # Compute CPU energy for baseline
            baseline_cpu_energy = (baseline_cpu_data["Package Power (W)"].mean() + 
                                   baseline_cpu_data["DRAM Power (W)"].mean()) * baseline_exec_time

            # Compute GPU energy for baseline
            baseline_gpu_energy = baseline_gpu_data["Power (W)"].mean() * baseline_exec_time

            # Total energy for baseline
            baseline_total_energy = baseline_cpu_energy + baseline_gpu_energy

            # Compute baseline EDP
            baseline_edp = baseline_total_energy * baseline_exec_time

            # Compute EDP for selected power cap pair
            selected_cpu_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_cpu_power.csv")
            selected_gpu_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_gpu_metric.csv")

            if os.path.exists(selected_cpu_file) and os.path.exists(selected_gpu_file):
                selected_cpu_data = pd.read_csv(selected_cpu_file)
                selected_gpu_data = pd.read_csv(selected_gpu_file)

                # Execution time from the last row of Time (s)
                selected_exec_time = selected_cpu_data["Time (s)"].iloc[-1]

                # Compute CPU energy for selected power cap
                selected_cpu_energy = (selected_cpu_data["Package Power (W)"].mean() + 
                                       selected_cpu_data["DRAM Power (W)"].mean()) * selected_exec_time

                # Compute GPU energy for selected power cap
                selected_gpu_energy = selected_gpu_data["Power (W)"].mean() * selected_exec_time

                # Total energy for selected power cap
                selected_total_energy = selected_cpu_energy + selected_gpu_energy

                # Compute EDP for selected power cap
                selected_edp = selected_total_energy * selected_exec_time

                # Store results
                edp_results[app] = {
                    "Best Power Pair": f"{cpu_power}_{gpu_power}",
                    "Baseline EDP": baseline_edp,
                    "Selected EDP": selected_edp
                }

hello


FileNotFoundError: [Errno 2] No such file or directory: './ecp_power_cap_res/2_dual_cap/bert_large/120_210_cpu_power.csv'

In [14]:
edp_results

{}

In [5]:
# Print the EDP results
for app, results in edp_results.items():
    print(f"Application: {app}")
    print(f"Best Power Pair: {results['Best Power Pair']}")
    print(f"Baseline EDP: {results['Baseline EDP']:.4f}")
    print(f"Selected EDP: {results['Selected EDP']:.4f}")
    print("-" * 40)


In [12]:
# Load performance matrix to check row indices type
performance_matrix = pd.read_csv("./prediction_res/performance_matrix.csv", index_col=0)

# Get index type and sample values
index_type = type(performance_matrix.index)
index_sample = performance_matrix.index[:5]  # Get first 5 indices for reference

index_type, index_sample


(pandas.core.indexes.base.Index,
 Index(['(120, 150)', '(120, 160)', '(120, 170)', '(120, 180)', '(120, 190)'], dtype='object'))